In [1]:
from sklearn.metrics import roc_auc_score
import torch
from torchvision.transforms import Compose, Normalize, Resize, ToTensor
from torch.optim import Adam
from torch.nn import BCELoss
import torch.utils.model_zoo as model_zoo
from cxrlib.models import guan_resnet_ag
from cxrlib.read_data import ChestXrayDataSet
from cxrlib.results import compute_AUCs
import numpy as np
from visdom import Visdom
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

In [3]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [4]:
# Run modified version of resnet50 on GPU
model = guan_resnet_ag.GuanResNet50_TwoAG().cuda()
model = torch.nn.DataParallel(model)
#model.load_my_state_dict(model_zoo.load_url(model_urls['resnet50']))
model.load_state_dict(torch.load('3'))
# Ensure that we can run the model on multiple GPUs
model.train()


DataParallel(
  (module): GuanResNet50_TwoAG(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
    

In [5]:
n_eps = 30
max_batches = 0
optimizer = Adam(model.parameters(), lr=0.0000005)
criterion = BCELoss()
normalize = Normalize([0.485, 0.456, 0.406],
                      [0.229, 0.224, 0.225])
# Utilize customized chest xray dataset loading mechanism. Add additional transforms. Downsize all
# images to 224 pixels, then convert to tensor, and finally normalize.
dataset = ChestXrayDataSet('/media/minh/UStorage/chestxray14/images',
                           '/media/minh/UStorage/chestxray14/labels/train_val_list.processed',
                           transform=Compose([Resize(224), ToTensor(), normalize]))
loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [6]:
log = open("2AG_00001/loss.txt","a")

In [ ]:
for ep in range(n_eps):

    for inp, target in loader:
        target = torch.autograd.Variable(target).cuda()
        inp = torch.autograd.Variable(inp).cuda()
        out = model(inp)
        optimizer.zero_grad()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
#        print(loss.cpu().detach().numpy().item())
        log.write(str(loss.cpu().detach().numpy().item()))
        log.write("\n")
#        loss_list.append(loss.cpu().detach().numpy().item())
#        if(counter % window == window-1):
#            try:
#                win = visdom_plot(viz,win,'./log.txt','test','test',window)
#            except IOError:
#                pass
#        counter += 1
    torch.save(model.state_dict(),str(ep+17))

/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/minh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
